In [1]:
DATA_NAME = 'syntheticMRI2D-coronal' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_vertical_wavelet_coronal_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,3,4,5,6,7,8,9
3,946850.55,22.08,-0.28,-5.57,0.61,-0.53,0.01
4,22.08,391888.27,9.43,1.52,3.11,-0.68,-0.04
5,-0.28,9.43,115589.73,0.53,-0.65,0.50,0.03
6,-5.57,1.52,0.53,35464.96,-0.39,-0.11,0.10
7,0.61,3.11,-0.65,-0.39,6775.12,0.04,0.00
8,-0.53,-0.68,0.50,-0.11,0.04,735.18,0.00
9,0.01,-0.04,0.03,0.10,0.00,0.00,65.57


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

,3,4,5,6,7,8,9
3,1.00000,0.00004,-0.00000,-0.00003,0.00001,-0.00002,0.00000
4,0.00004,1.00000,0.00004,0.00001,0.00006,-0.00004,-0.00001
5,-0.00000,0.00004,1.00000,0.00001,-0.00002,0.00005,0.00001
6,-0.00003,0.00001,0.00001,1.00000,-0.00002,-0.00002,0.00007
7,0.00001,0.00006,-0.00002,-0.00002,1.00000,0.00002,0.00001
8,-0.00002,-0.00004,0.00005,-0.00002,0.00002,1.00000,0.00000
9,0.00000,-0.00001,0.00001,0.00007,0.00001,0.00000,1.00000


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

35.26056583578886

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.28858465e+11 2.13018836e+10 1.78591301e+09 1.56811515e+08
 5.09226214e+06 4.52625037e+04 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.996412,-0.082078,-0.019754,-0.005971,-0.001138,-0.000124,-0.000011
1,0.080490,0.994394,-0.065992,-0.018317,-0.003458,-0.000378,-0.000034
2,0.024316,0.062375,0.993997,-0.085302,-0.014451,-0.001557,-0.000139
3,0.009396,0.022798,0.082651,0.993977,-0.067252,-0.006947,-0.000616
4,0.002383,0.005750,0.019573,0.065225,0.996115,-0.055269,-0.004843
5,0.000385,0.000929,0.003143,0.010263,0.054004,0.994486,-0.089241
6,0.000069,0.000167,0.000564,0.001839,0.009638,0.088833,0.995998


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0035884265628040035,
 0.005606282168073129,
 0.00600281317038498,
 0.006023473635171306,
 0.0038846895807221893,
 0.0055135722922269315,
 0.004001953730866981]